In [1]:
from arg_utils import is_notebook, get_cfg
cfg = get_cfg()
# override variables to experiment in notebook
if is_notebook():
    cfg["gpu"] = 2
    cfg["debug"] = False
    
    cfg["model_name"] = "ViT-B/16"    
    cfg["mode"] = "adapters" # "adapters", "train_mlp_norm", "full"
    cfg["mixup_alpha"] = 0.5
    
    cfg["text2img_num_feats"] = 2
    
# overrides and calculated default vals
if cfg["batch_size"] is None:
    if cfg["model_name"] == "ViT-L/14":
        batch_size = 4 # max 32 for single GPU CL on VitB16, 4 for ViT-L/14 (9.2GB)
    elif cfg["model_name"] == "ViT-B/16":
        batch_size = 32
    elif cfg["model_name"] == "ViT-B/32":
        batch_size = 64
    else:
        batch_size = 32
    cfg["batch_size"] = batch_size
cfg["val_check_interval"] = int(cfg["val_check_interval"] * (32 / cfg["batch_size"]))
    

# seed everything using pytorch lightning
import pytorch_lightning
pytorch_lightning.seed_everything(cfg["seed"])

locals().update(cfg)

Global seed set to 0


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = str(cfg["gpu"])

import pandas as pd
from PIL import Image
import numpy as np

import torch

root_folder = "/raid/8wiehe/"

In [4]:
from contrastive_learning_utils import init_dm, init_lit_model, init_clip_model, init_trainer, init_test_dms

clip_base_model, transform, clip_name = init_clip_model(cfg)
data_module = init_dm(cfg["dataset_name"], root_folder, clip_base_model, transform, cfg, use_cl=True, use_augs=cfg["use_augs"], batch_size=cfg["batch_size"])
test_data_modules = init_test_dms(["covidx"], transform, cfg)

lit_model = init_lit_model(clip_base_model, test_data_modules, data_module.steps_per_epoch, data_module.label_names, cfg)
trainer = init_trainer(root_folder, cfg)
trainer.fit(lit_model, data_module)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


602496 149556226 0.004028558463356785


wandb: Currently logged in as: antonius (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [2]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | CLIP             | 150 M 
1 | loss_func | CrossEntropyLoss | 0     
-----------------------------------------------
602 K     Trainable params
149 M     Non-trainable params
150 M     Total params
300.317   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 0


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]